In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import HuggingFaceHub
import pickle

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
loader = UnstructuredURLLoader(
    urls=[
        "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
        "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
    ]
)
docs = loader.load()

In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [5]:
spltted_docs = splitter.split_documents(docs)

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


C:\Users\SUBHRADEEP\AppData\Local\Temp\ipykernel_5160\614204990.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\SUBHRADEEP\Langchain\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SUBHRADEEP\Langchain\myenv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\

In [7]:
vector_store = FAISS.from_documents(spltted_docs, embeddings)

In [8]:
with open("vector_store.pkl", "wb") as f:
    pickle.dump(vector_store, f)

In [9]:
with open("vector_store.pkl", "rb") as f:
    loaded_vector_store = pickle.load(f)

In [10]:
retriever = loaded_vector_store.as_retriever()

In [13]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    max_tokens=100,
    temperature=0.5,
)

In [14]:
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    retriever=retriever
)

In [17]:
query = "what is the price of Tiago iCNG?"
result = qa_chain(query)
print("Answer:", result["answer"])
print("Sources:", result["sources"])

Answer: The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.


Sources: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html
